**Where is this data set from?**

- The World Happiness Report is an annual publication of the United Nations Sustainable Development Solutions Network. This dataset is a subset of the larger report, which includes data from various sources such as the Gallup World Poll and other national surveys. The data was extracted from the World Happiness Report and made available for public use. However, the original data was collected by various researchers and organizations as part of their ongoing efforts to measure and understand happiness and well-being around the world.

    We use observed data on the six variables and estimates of their associations with life evaluations to explain the variation across countries. They include GDP per capita, social support, healthy life expectancy, freedom, generosity, and corruption. Our happiness rankings are not based on any index of these six factors – the scores are instead based on individuals’ own assessments of their lives, in particular, their answers to the single-item Cantril ladder life-evaluation question, much as epidemiologists estimate the extent to which life expectancy is affected by factors such as smoking, exercise, and diet

Detailed information about each of the Predictors:

1. **Log GDP per capita** is in terms of Purchasing Power Parity (PPP) adjusted to a constant 2017 international dollars, taken from the World Development Indicators (WDI) by the World Bank (version 17, metadata last updated on January 22, 2023). See Statistical Appendix 1 for more details. GDP data for 2022 are not yet available, so we extend the GDP time series from 2021 to 2022 using country-specific forecasts of real GDP growth from the OECD Economic Outlook No. 112 (November 2022) or, if missing, from the World Bank’s Global Economic Prospects (last updated: January 10, 2023), after adjustment for population growth. The equation uses the natural log of GDP per capita, as this form fits the data significantly better than GDP per capita.

2. The time series for **Healthy life expectancy at birth** is constructed based on data from the World Health Organization (WHO) Global Health Observatory data repository, with data available for 2005, 2010, 2015, 2016, and 2019. To match this report’s sample period (2005-2022), interpolation and extrapolation are used. See Statistical Appendix 1 for more details.

3. **Social support** (0-1) is the national average of the binary responses (0=no, 1=yes) to the Gallup World Poll (GWP) question “If you were in trouble, do you have relatives or friends you can count on to help you whenever you need them, or not?”

4. **Freedom to make life choices** (0-1) is the national average of binary responses to the GWP question “Are you satisfied or dissatisfied with your freedom to choose what you do with your life?”

5. **Generosity** is the residual of regressing the national average of GWP responses to the donation question “Have you donated money to a charity in the past month?” on log GDP per capita.

6. **Perceptions of corruption** (0-1) are the average of binary answers to two GWP questions: “Is corruption widespread throughout the government or not?” and “Is corruption widespread within businesses or not?” Where data for government corruption are missing, the perception of business corruption is used as the overall corruption perception measure.

7. **Positive affect** is defined as the average of previous-day effects measures for laughter, enjoyment, and interest. The inclusion of interest (first added for World Happiness Report 2022), gives us three components in each of positive and negative affect, and slightly improves the equation fit in column 4. The general form for the affect questions is: Did you experience the following feelings during a lot of the day yesterday?

8. **Negative affect** is defined as the average of previous-day effects measures for worry, sadness, and anger.

9. **Life ladder**: Life evaluations from the Gallup World Poll provide the basis for the annual happiness rankings. They are based on answers to the main life evaluation question. The Cantril ladder asks respondents to think of a ladder, with the **best possible life for them being a 10 and the worst possible life being a 0**. They are then asked to rate their own current lives **on a 0 to 10 scale**. The rankings are from nationally representative samples over three years.

10. **Confidence in National Government**: The "Confidence in National Government" variable in the World Happiness Report is calculated based on the following question asked in the Gallup World Poll:

    "Do you have confidence in the national government?"

    Respondents are given the following options to choose from:

    "Yes, always"
    
    "Yes, sometimes"
    
    "No, rarely"

    "No, never"

    "Don't know"

    **The variable is calculated as the percentage of respondents who answer "Yes, always" or "Yes, sometimes" to this question.**

    This variable is one of several social factors that are included in the calculation of the World Happiness Report's overall happiness score for each country. The report combines data on social factors such as income, social support, life expectancy, freedom to make life choices, generosity, and perceptions of corruption to arrive at a comprehensive measure of happiness.

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames


In [2]:
from src.tolookandcompare import transform_headtail, transform_info

In [3]:
from src import soporte_eda as sp_eda

In [4]:
df = pd.read_csv ('World Happiness Report.csv')

df.head(2)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545


In [5]:
df.sample(10)

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
1156,Luxembourg,Western Europe,2010,7.097252,11.646967,0.952372,70.500000,0.908303,0.086880,0.423341,0.717743,0.216064,0.768445
495,Czechia,NaN,2012,6.334149,10.424900,0.912427,67.940002,0.739809,-0.157698,0.956800,0.634971,0.256508,0.169811
1868,Switzerland,Western Europe,2012,7.776209,11.107383,0.946864,71.519997,0.945428,0.132033,0.323241,0.792705,0.176007,0.769940
1428,Nicaragua,Latin America and Caribbean,2022,6.392258,8.664050,0.844042,65.724998,0.913994,-0.001198,0.570192,0.787344,0.338744,NaN
1644,Rwanda,Sub-Saharan Africa,2016,3.332990,7.553644,0.665131,59.224998,0.910736,0.025161,0.158601,0.714600,0.285384,0.993604
2012,Uganda,Sub-Saharan Africa,2012,4.309238,7.607653,0.884722,53.779999,0.649463,0.078395,0.837546,0.708793,0.265322,0.396223
1010,Kazakhstan,Commonwealth of Independent States,2019,6.272268,10.179292,0.951050,65.000000,0.852387,-0.058804,0.708279,0.711475,0.139133,0.671793
1558,Philippines,Southeast Asia,2013,4.976925,8.796468,0.846413,61.779999,0.907458,0.019095,0.756389,0.795775,0.331958,0.751475
2163,Yemen,Middle East and North Africa,2016,3.825631,7.552322,0.775407,58.174999,0.532964,-0.140766,NaN,0.401007,0.227925,0.267581
409,Colombia,Latin America and Caribbean,2018,5.983512,9.576818,0.870970,68.824997,0.850766,-0.151604,0.854821,0.774538,0.300624,0.274870


In [6]:
transform_headtail(df, 'Country Name')

Valores únicos: 165
Número de registros: 2199
Valores nulos: 0
Registros duplicados: 2034
dtype: object
---------------------------------
Country Name
Lebanon       0.77
Indonesia     0.77
Kyrgyzstan    0.77
Kenya         0.77
Kazakhstan    0.77
Name: proportion, dtype: float64
Country Name
Maldives    0.05
Guyana      0.05
Oman        0.05
Suriname    0.05
Cuba        0.05
Name: proportion, dtype: float64
---------------------------------
Media: nan
Mediana: nan
Moda: nan


In [7]:
transform_headtail(df, 'Generosity')

Valores únicos: 2126
Número de registros: 2199
Valores nulos: 73
Registros duplicados: 73
dtype: float64
---------------------------------
Generosity
 NaN         3.32
 0.167652    0.05
-0.046418    0.05
-0.011639    0.05
 0.021866    0.05
Name: proportion, dtype: float64
Generosity
-0.193871    0.05
 0.117222    0.05
 0.108049    0.05
 0.198711    0.05
-0.069513    0.05
Name: proportion, dtype: float64
---------------------------------
Media: 0.00
Mediana: -0.02
Moda: -0.33752656


In [8]:
sp_eda.resumen_df(df)


Forma del DataFrame: (2199, 13)

Tipos de datos:
Country Name                         float64
Regional Indicator                    object
Year                                   int64
Life Ladder                          float64
Log GDP Per Capita                   float64
Social Support                       float64
Healthy Life Expectancy At Birth     float64
Freedom To Make Life Choices         float64
Generosity                           float64
Perceptions Of Corruption            float64
Positive Affect                      float64
Negative Affect                      float64
Confidence In National Government    float64
dtype: object

Valores nulos:
Country Name                         2199
Regional Indicator                    112
Year                                    0
Life Ladder                             0
Log GDP Per Capita                     20
Social Support                         13
Healthy Life Expectancy At Birth       54
Freedom To Make Life Choices           33

,count,mean,std,min,25%,50%,75%,max
Country Name,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year,2199.0,2014.161437,4.718736,2005.000000,2010.000000,2014.000000,2018.000000,2022.000000
Life Ladder,2199.0,5.479226,1.125529,1.281271,4.646750,5.432437,6.309460,8.018934
Log GDP Per Capita,2179.0,9.389766,1.153387,5.526723,8.499764,9.498955,10.373216,11.663788
Social Support,2186.0,0.810679,0.120952,0.228217,0.746609,0.835535,0.904792,0.987343
Healthy Life Expectancy At Birth,2145.0,63.294583,6.901104,6.720000,59.119999,65.050003,68.500000,74.474998
Freedom To Make Life Choices,2166.0,0.747858,0.140150,0.257534,0.656528,0.769821,0.859382,0.985178
Generosity,2126.0,0.000096,0.161083,-0.337527,-0.112116,-0.022671,0.092070,0.702708
Perceptions Of Corruption,2083.0,0.745195,0.185837,0.035198,0.688139,0.799654,0.868827,0.983276
Positive Affect,2175.0,0.652143,0.105922,0.178886,0.571684,0.663063,0.737936,0.883586



Resumen estadístico (categóricas):


,count,unique,top,freq
Regional Indicator,2087,10,Sub-Saharan Africa,443


In [9]:
df['Country Name'].dtype

dtype('float64')

In [11]:
df['Country Name']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2194   NaN
2195   NaN
2196   NaN
2197   NaN
2198   NaN
Name: Country Name, Length: 2199, dtype: float64

In [12]:
df["Country Name"] = df["Country Name"].astype(str)

In [13]:
df['Country Name']

0       nan
1       nan
2       nan
3       nan
4       nan
       ... 
2194    nan
2195    nan
2196    nan
2197    nan
2198    nan
Name: Country Name, Length: 2199, dtype: object

In [10]:
# df['Country Name'].astype('O')

____

Insights: 

- Hay que pasar la columna 'Country' de float a objeto. Gran número de nulos por este motivo. 